In [1]:
import numpy as np 
from astropy.io import fits
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib ipympl

In [2]:
dgroup = np.loadtxt('../odata/CLAUDS_HSC_iband_group')
digal = np.loadtxt('../odata/CLAUDS_HSC_iband_igal')
d2 = np.loadtxt('../odata/iCLAUDS_HSC_iband_2')
d1 = np.loadtxt('../odata/iCLAUDS_HSC_iband_1')

In [5]:
#recode id for reference sample
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
from astropy.coordinates import SkyCoord
cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315)

idx_sel = np.where((dgroup[:,-2] > 14) & (dgroup[:,4] < 0.3))[0] #select with Mh > 1e14 and z < 0.1
clu_id = np.int64(dgroup[idx_sel,0])
Nc = idx_sel.shape[0]
Nsat = np.zeros(Nc, dtype = np.int64)

referen_id = []
for i in range(Nc):
    cid = np.int64(dgroup[idx_sel[i],0])
    credz = dgroup[idx_sel[i],4]
    cra = dgroup[idx_sel[i],2]
    cdec = dgroup[idx_sel[i],3]
    cn = dgroup[idx_sel[i],1]

    idx_gal = np.where(d2[:,0] == cid)[0]
    galid = np.int64(d2[idx_gal,1])
    memra = digal[galid-1,1]
    memdec = digal[galid-1,2]
    memredz = digal[galid-1,3]
    d_r = cal_rp(memredz,credz,memra,memdec,cra,cdec)
    edge_r = np.max(d_r) #choose the most distant member distance

#         import linecache
#         fprop = '../odata/CLAUDS_HSC_iband_prop'
#         nline = 1+(cid-1)*6
#         gprop = linecache.getline(fprop,nline)
#         phyp = [float(prop) for prop in gprop.split()]
#         r180 = phyp[-2] #unit:Mpc/h

    idx_ag = np.where((np.abs(digal[:,1] - cra) < 0.7) & (np.abs(digal[:,2] - cdec) < 0.5) &
                     (np.abs(digal[:,3] - credz) < 0.1))[0] 
    ag_abm = digal[idx_ag,5] #absolute magnitude of galaxies
    ag_ra = digal[idx_ag,1]
    ag_dec = digal[idx_ag,2] 
    ag_redz = digal[idx_ag,3]
    ag_id = np.int64(digal[idx_ag,0])
    d_r = cal_rp(ag_redz,credz,ag_ra,ag_dec,cra,cdec)

    idx_inr = np.where(d_r < edge_r)[0]
    referen_id.append(ag_id[idx_inr])

In [4]:
def cal_rp(ag_redz,c_redz,ag_ra,ag_dec,c_ra,c_dec):
    from astropy.cosmology import FlatLambdaCDM
    from astropy import units as u
    from astropy.coordinates import SkyCoord
    cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315)
    
    dr_gal = cosmo.comoving_distance(ag_redz).value * 0.674 #Mpc/h
    dr_clu = cosmo.comoving_distance(c_redz).value * 0.674 #Mpc/h

    cgal = SkyCoord(ra=ag_ra*u.degree, dec=ag_dec*u.degree, distance=dr_gal)
    cclu = SkyCoord(ra=c_ra*u.degree, dec=c_dec*u.degree, distance=dr_clu)
    cgal_x = cgal.cartesian.x
    cgal_y = cgal.cartesian.y
    cgal_z = cgal.cartesian.z

    cclu_x = cclu.cartesian.x
    cclu_y = cclu.cartesian.y
    cclu_z = cclu.cartesian.z

    l = np.array([cgal_x+cclu_x, cgal_y+cclu_y, cgal_z+cclu_z]).T / 2
    s = np.array([cclu_x - cgal_x, cclu_y - cgal_y, cclu_z - cgal_z]).T
    r_pi = np.sum(l*s,axis = 1) / np.sqrt(np.sum(l**2, axis = 1)) 
    r_p = np.sqrt(np.sum(s**2,axis = 1) - r_pi**2)
    
    return r_p

In [6]:
def cal_Nsat(redz):  
    from astropy.cosmology import FlatLambdaCDM
    from astropy import units as u
    from astropy.coordinates import SkyCoord
    cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315)
        
    idx_sel = np.where((dgroup[:,-2] > 14) & (dgroup[:,4] < 0.3))[0] #select with Mh > 1e14 and z < 0.1
    clu_id = np.int64(dgroup[idx_sel,0])
    Nc = idx_sel.shape[0]
    Nsat = np.zeros(Nc, dtype = np.int64)
    for i in range(Nc):
        gal_abm = digal[referen_id[i]-1,5]

        d_L = cosmo.luminosity_distance(redz).value * 0.674
        Mc = 26 - 5*np.log10(d_L) - 25

        Nsat[i] = np.where(gal_abm < Mc)[0].shape[0]
    print('z = %s: min %s, mean %s' %(redz, np.min(Nsat), np.mean(Nsat)))
    return Nsat

In [6]:
# gid = [16586, 20048, 57726]

cal_Nsat(4)

z = 4: min 26, mean 56.111111111111114


array([ 54,  80,  89, 109,  83, 170,  68,  75,  34,  61,  52,  45,  78,
        48,  58,  51,  58,  37,  38,  42,  38,  32,  39,  62,  74,  29,
        36,  26,  37,  39,  70,  39,  62,  42,  33,  32])

In [7]:
#test for one sample
Nc = idx_sel.shape[0]
Nsat = np.zeros((50,Nc), dtype = np.int64)
zseq = np.linspace(2,4.3,50)
for k in range(50):
    choose_z = zseq[k]
    Nsat0 = cal_Nsat(choose_z)
    Nsat[k] = Nsat0
    print(k)


# for i in range(Nc):
#     cid = np.int64(dgroup[idx_sel[i],0])
#     credz = dgroup[idx_sel[i],4]
#     idx_gal = np.where(d2[:,0] == cid)[0]
#     gid = np.int64(d2[idx_gal,1])

# #     gal_redz = digal[gid-1,3]
# #     gal_mag = digal[gid-1,4]
#     gal_abm = digal[gid-1,5] #absolute magnitude of galaxies

#     from astropy.cosmology import FlatLambdaCDM
#     from astropy import units as u
#     cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315)

#     d_L = cosmo.luminosity_distance(choose_z).value * 0.674
#     Mc = 26 - 5*np.log10(d_L) - 25
#     Ncho[k,i] = gal_abm.shape[0]

z = 2.0: min 180, mean 345.3333333333333
0
z = 2.046938775510204: min 173, mean 332.3333333333333
1
z = 2.093877551020408: min 167, mean 319.19444444444446
2
z = 2.1408163265306124: min 161, mean 307.25
3
z = 2.1877551020408164: min 157, mean 294.8888888888889
4
z = 2.2346938775510203: min 149, mean 283.8888888888889
5
z = 2.2816326530612243: min 142, mean 273.55555555555554
6
z = 2.3285714285714283: min 138, mean 263.02777777777777
7
z = 2.3755102040816327: min 135, mean 252.58333333333334
8
z = 2.4224489795918367: min 128, mean 242.16666666666666
9
z = 2.4693877551020407: min 118, mean 232.69444444444446
10
z = 2.516326530612245: min 113, mean 224.08333333333334
11
z = 2.5632653061224486: min 108, mean 215.13888888888889
12
z = 2.610204081632653: min 101, mean 207.19444444444446
13
z = 2.657142857142857: min 98, mean 199.38888888888889
14
z = 2.704081632653061: min 93, mean 191.25
15
z = 2.7510204081632654: min 90, mean 183.88888888888889
16
z = 2.7979591836734694: min 87, mean 177.2

In [9]:
plt.figure(figsize = (5.3,4.0))

import matplotlib as mpl
mpl.rcParams['axes.linewidth'] = 1.2 #set the value globally

plt.plot(zseq, np.min(Nsat, axis = 1), c = 'k', lw = 2, label = 'minimum')
plt.plot(zseq, np.median(Nsat, axis = 1), c = 'k', ls = '--',label = 'median')
plt.plot(zseq, np.mean(Nsat, axis = 1), c = 'k', lw = 2, ls = ':', label = 'mean')

plt.xlabel(r'$z$', fontsize = 14)
plt.ylabel(r'$N_{\rm cri}$', fontsize = 14)
plt.xlim(2,4.3)

plt.tick_params(top = 'on', right = 'on', which='both',direction = 'in',labelsize = 12)

plt.text(2.35, 50, 'Low', fontsize = 16)
plt.text(2.7, 110, 'Medium', fontsize = 16)
plt.text(3.3, 220, 'High', fontsize = 16)

plt.legend(fontsize = 'large')
plt.tight_layout()
plt.savefig('../figs/Ncri_z.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# idx_sel = np.where((dgroup[:,-2] > 14) & (dgroup[:,4] < 0.3))[0] #select with Mh > 1e14 and z < 0.1
# clu_id = np.int64(dgroup[idx_sel,0])
# sigma_z = np.zeros(clu_id.shape[0])
# for i in range(clu_id.shape[0]):
#     idx_gal = np.where(d2[:,0] == clu_id[i])[0]
#     gid = np.int64(d2[idx_gal,1])
#     redz_gal = digal[gid-1,3]
#     sigma_z[i] = np.std(redz_gal)

In [37]:
plt.figure()

plt.plot(Ncho, Nsat, 'o')
plt.axhline(np.mean(Nsat), c = 'grey', ls = '--', label = 'mean')
plt.axhline(np.median(Nsat), c = 'grey', ls = ':', label = 'median')

plt.xlabel(r'$N_{\rm g}$')
plt.ylabel(r'$N_{\rm satg}$')
plt.title('z = 4')

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
def cal_Nsat(redz):  
    from astropy.cosmology import FlatLambdaCDM
    from astropy import units as u
    from astropy.coordinates import SkyCoord
    cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315)
        
    idx_sel = np.where((dgroup[:,-2] > 14) & (dgroup[:,4] < 0.3))[0] #select with Mh > 1e14 and z < 0.1
    clu_id = np.int64(dgroup[idx_sel,0])
    Nc = idx_sel.shape[0]
    Nsat = np.zeros(Nc, dtype = np.int64)
    plt.figure()
    for i in range(1):
        i = 3
        cid = np.int64(dgroup[idx_sel[i],0])
        credz = dgroup[idx_sel[i],4]
        cra = dgroup[idx_sel[i],2]
        cdec = dgroup[idx_sel[i],3]
        cn = dgroup[idx_sel[i],1]
        
        import linecache
        fprop = '../odata/CLAUDS_HSC_iband_prop'
        nline = 1+(cid-1)*6
        gprop = linecache.getline(fprop,nline)
        phyp = [float(prop) for prop in gprop.split()]
        r180 = phyp[-2] #unit:Mpc/h
        
        idx_gal = np.where(d2[:,0] == cid)[0]
        galid = np.int64(d2[idx_gal,1])
        memra = digal[galid-1,1]
        memdec = digal[galid-1,2]
                
        cdud = SkyCoord(cra, cdec, unit="deg")
        cc = SkyCoord(memra, memdec, unit="deg")
        sep = cdud.separation(cc)
        d_A = cosmo.comoving_distance(credz) * 0.674 #Mpc/h
        d_r = (sep * d_A).to(u.Mpc, u.dimensionless_angles()).value #to be a comoving distance
        print(np.max(d_r))
        idx = np.argmax(d_r)
        plt.plot(memra[idx],memdec[idx],'*')
        plt.plot(memra,memdec,'s',mfc = 'none')
        
        idx_ag = np.where((np.abs(digal[:,1] - cra) < 0.7) & (np.abs(digal[:,2] - cdec) < 0.5) &
                         (np.abs(digal[:,3] - credz) < 0.1))[0] 
        ag_abm = digal[idx_ag,5] #absolute magnitude of galaxies
        ag_ra = digal[idx_ag,1]
        ag_dec = digal[idx_ag,2]
        plt.plot(ag_ra,ag_dec,'.',ms = 2)
        
        cdud = SkyCoord(cra, cdec, unit="deg")
        cc = SkyCoord(ag_ra, ag_dec, unit="deg")
        sep = cdud.separation(cc)
        d_A = cosmo.comoving_distance(credz) * 0.674 #Mpc/h
        d_r = (sep * d_A).to(u.Mpc, u.dimensionless_angles()).value #to be a comoving distance
        print(r180, np.max(d_r), idx_ag.shape)
        idx_inr = np.where(d_r < r180)[0]
        print(cn, idx_inr.shape[0])
        gal_abm = ag_abm[idx_inr]
        plt.plot(ag_ra[idx_inr],ag_dec[idx_inr],'s', ms = 2)

        d_L = cosmo.luminosity_distance(redz).value * 0.674
        Mc = 26 - 5*np.log10(d_L) - 25

        Nsat[i] = np.where(gal_abm < Mc)[0].shape[0]
#     print('z = %s: min %s, mean %s' %(redz, np.min(Nsat), np.mean(Nsat)))
    return Nsat

In [76]:
cal_Nsat(2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

4.475347021739458
2.192 7.32893613016899 (15200,)
4619.0 2965


array([  0,   0,   0, 279,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0])